# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [172]:
# import libraries here; add more as necessary
import time
import numpy as np
import pandas as pd
import random
import re
import progressbar
from timeit import default_timer as timer

from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.externals import joblib
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold

import xgboost as xgb
import lightgbm as lgb

from utils import *

import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 1: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [2]:
# display more rows and columns
def display_all(df, rows=100):
    """
    Displays 1000 rows and columns
    
    Args:
        df (DataFrame): DataFrame to display
    """
    with pd.option_context("display.max_rows", rows, "display.max_columns", 1000):
        display(df)

In [3]:
# load in the data and use string for column 18 and 19
#azdias = pd.read_csv('data/Udacity_AZDIAS_052018.csv', sep=';', dtype={18:'str',19:'str'})
#customers = pd.read_csv('data/Udacity_CUSTOMERS_052018.csv', sep=';', dtype={18:'str',19:'str'})

In [4]:
# save to pickle
#azdias.to_pickle('data/azdias.pkl')
#customers.to_pickle('data/customers.pkl')

In [11]:
# load pickle file
azdias = pd.read_pickle('data/azdias.pkl')
customers = pd.read_pickle('data/customers.pkl')

In [ ]:
print(f'General population (azidas) has {azdias.shape[0]} rows and {azdias.shape[1]} columns')
print(f'Customer population (customers) has {customers.shape[0]} rows and {customers.shape[1]} columns')

Looks like the datasets were loaded correctly. Lets begin this exploration phase with a look at the summary statistics for all continuous variables as well as the first couple of rows.

In [ ]:
azdias.head(2)

In [ ]:
customers.head(2)

One thing that is pretty obvious by just looking at the tables above is that some columns seem to contain a lot of NaN:s. The descriptive summary below shows this as well.

The amount of missing values will increase after converting missing value codes/unknowns to NaN:s

In [ ]:
azdias.describe()

## 1.1 Preprocessing
In this section I'll investigate and handle missing values, re-encode features, some feature engineering as well as feature scaling.

In [8]:
feat_info = pd.read_csv('data/feat_final.csv', sep=',')
feat_info.drop('Unnamed: 0', axis=1, inplace=True)
feat_info.head()

,attribute,information_level,type,missing_or_unknown
0,AGER_TYP,person,categorical,"[-1, 0]"
1,ALTERSKATEGORIE_GROB,person,ordinal,"[-1, 0, 9]"
2,ANREDE_KZ,person,categorical,"[-1, 0]"
3,CJT_GESAMTTYP,person,categorical,[0]
4,FINANZ_MINIMALIST,person,ordinal,[-1]


The features information file loaded above contains valuable information about each feature. This file will be used throughout the project to help decide what needs to be done and how to do it.

The feature file is a combination of the old feature file from the unsupervised project in term 1 and the new feature file. To create the feature file used in this project I had to do the following:

- Manually scan through the feature and the attribute file provided for this project and:
	- Assign feature type
	- Assign missing or unknown codes
        - The values for ``'unknown'``, ``'unknown / no main age detectable'``, ``'no transactions known'`` or ``'no transaction known'`` where used to define these codes.
- Check for contradicting information
	- Some features that were found in both the file from term 1 and the one provided for this project have different values for the missing or unknown column.  One example of this is the codes for the feature ``CAMEO_DEUG_2015``. The old feature file states that ``[-1, X]`` should be treated as missing while the new only contains ``[-1]``. For cases like this I decided to go with the values from the file from term 1. The reason for this is simply that this information is corresponding with the values in the dataset better.
- Convert all values in the missing_or_unknown column to abide to the following format:
    - [int] --> [1]
    - [int, int] --> [-1, 0]
    - [int, 'str'] --> [1, 'XX']
    - [] --> []

In [ ]:
azdias['CAMEO_DEUG_2015'].value_counts()

Even though most of the creation of the feature file went smooth some features (57 to be exact) found in the dataset could not be found in any of the provided feature files. This is a problem due to the fact that we have no information about what type of feature these are or what values in those columns that should be treated as missing. I decided to treat all these values as ordinal and assigned an empty array to the missing or unknown column. Another solution would be to just drop all these features.

### 1.1.1 Assess Missing Data

In [ ]:
# missing values per column before converting missing codes
missing_values_unparsed = azdias.isnull().sum()

In [ ]:
# convert missing value codes to NaN
azdias_parsed = convert_missing_codes(azdias, feat_info)

In [ ]:
# missing values per column after converting missing codes
missing_values_parsed = azdias_parsed.isnull().sum()

In [ ]:
missing_before_total = missing_values_unparsed.sum()
missing_after_total = missing_values_parsed.sum()
diff = missing_after_total - missing_before_total

print(f'Number of missing values before converstion: {missing_before_total}')
print(f'Number of missing values after converstion: {missing_after_total}')
print('Increase in missing values: {0:.2f} % '.format(diff / missing_before_total * 100))

In [ ]:
# sorting so feat_info.... is in the same order as other dfs
sorter = missing_values_unparsed.index
sorterIndex = dict(zip(sorter, range(len(sorter))))
feat_info_final_sorted = feat_info.copy()
feat_info_final_sorted['feature_id_rank'] = feat_info_final_sorted['attribute'].map(sorterIndex)
feat_info_final_sorted.sort_values('feature_id_rank', ascending=True, inplace=True)
feat_info_final_sorted = feat_info_final_sorted.drop('feature_id_rank', axis=1)

In [ ]:
df_missing_values = pd.DataFrame(
    {
        'Before': missing_values_unparsed,
        'After': missing_values_parsed,
        'Diff': missing_values_parsed - missing_values_unparsed,
        'Percentage_Missing': missing_values_parsed / azdias.shape[0] * 100,
        'Information_Level': feat_info_final_sorted['information_level'].values
    })

df_missing_values.sort_values(["After", "Diff"], ascending=False).head()

As shown by the table above it's clear that some features have a lot more missing values after converting the missing value codes to NaN:s. Many of the columns that got a substantial increase in NaN's are belonging to ``D19`` (transactional). One thing that they have in common is that they have either "0" or "10" as missing value codes. In the documentation, these values are classified as "no transactions known" and as pointed out earlier I classified this as meaning missing or unknown.

There are probably few columns that are outliers in terms of the proportion of values that are missing. Matplotlib's hist() function will be used visualize the distribution of missing value counts to find these columns.

In [ ]:
fig = plt.figure(figsize=(20,7))
plt.hist(df_missing_values['Percentage_Missing'], bins=np.linspace(0,100,21))
plt.title('Distribution of Missing Values per Feature');
plt.xticks(np.linspace(0,100,21))
plt.xlabel('% Missing Values')
plt.ylabel('Number of Features')
plt.grid()

In [ ]:
less_30 = df_missing_values[df_missing_values['Percentage_Missing'] <= 30]['Percentage_Missing']

fig = plt.figure(figsize=(20,7))
less_30.hist()
plt.title('Distribution of Missing Values per Column <= to 30%');
plt.xlabel('% Missing Values')
plt.ylabel('Number of Features');

In [ ]:
missing_outliers = df_missing_values[df_missing_values['Percentage_Missing'] > 30]
print(f'{missing_outliers.shape[0]} features have more than 30 % missing values.')

Before removing any features lets see if anything interesting can be observed by plotting the amount of missing values per feature grouped by information_level.

In [ ]:
def plot_information_levels(df):
    """
    Plot every feature within each information_level group.
    
    Args:
        df (dataframe) - Dataframe to be used
    """
    sns.set(style='whitegrid')
    sns.set_color_codes('pastel')
    fig = plt.figure(figsize=(15,10))
    information_level = df['Information_Level'].unique()
    for idx, level in enumerate(information_level):
        sns.set()
        fig, ax = plt.subplots()
        sns.set(style="ticks")
        features = df[df['Information_Level'] == level].sort_values(['After'], ascending=False)
        ax = sns.barplot(x=features.index.values, y='After', data=features)
        ax.set(xlabel="Feature", ylabel="# Missing Values", title=level.title())
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
        fig.set_size_inches(22,7)
            

plot_information_levels(df_missing_values)

Group features by number of missing values and print groups that contain more than one feature.

In [ ]:
# create dict key = # missing values and value = array of features
missing_sorted = missing_values_parsed.sort_values()
missing_groups = dict()
for feat, val in missing_sorted.items():
    if val in missing_groups:
        missing_groups[val].append(feat)
    else:
        missing_groups[val] = [feat]

for val, feat in missing_groups.items():
    if len(feat) > 1:     
        print(f'These features share {val} missing value:\n {feat} \n')

By analyzing the plots and output above I found that features similar features or features that describe similar information but at different scales often have the same amount of missing values. Let's take ``LP_FAMILIE_FEIN`` and ``LP_FAMILIE_GROB`` for example.

In [ ]:
df_missing_values.loc[['LP_FAMILIE_FEIN', 'LP_FAMILIE_GROB']]

Both describe the person's family type, but at different scales. Fein translates to fine and Grob to rough. It's also stated that values in ``LP_FAMILIE_FEIN`` map to values in ``LP_FAMILIE_GROB``.

### 1.1.3 Removing Columns

The analysis have shown that the distribution of missing values per feature are skewed to the right meaning most features have zero or a relative small amount of missing values. The majority of the features have less than 30 % missing values.

In [ ]:
features_top_missing = [x for x in missing_outliers.index]
display_all(df_missing_values[df_missing_values.index.isin(features_top_missing)].sort_values('Percentage_Missing', ascending=False))

In [ ]:
old_feat_file = pd.read_csv('data/AZDIAS_Feature_Summary.csv', sep=';')

In [ ]:
new_exclusive = np.setdiff1d(features_top_missing, old_feat_file['attribute'])

print(f'Features with more than 30 % missing values originating from old feature file: {len(features_top_missing) - len(new_exclusive)}')
print(f'Features with more than 30 % missing values originating from new feature file: {len(new_exclusive)}')

No clear pattern can be observed for the features with the largest amount of missing values. Many of the features that got a large increase in missing values after the conversion are classified as either household or 125x125_grid. Another observation is that 60 out of the 66 features are from the new feature file. Why this is the case is hard to answer, but it could be because I manually parsed the missing code values (most are either 0 or 10).

Deciding which features to remove from the analysis is difficult. Rather than just using a threshold we should also try to take into account the amount of information we think the features contain. Another important question to ask is why do we have missing values? Is it due to errors, data gathering technique or do we have missing at random, missing completely at random or missing not at random values?

The sparse information regarding each feature in combination with the lack of domain knowledge as well as not knowing the German language makes this almost impossible and for that reason I have decided to use a threshold of 30 %.

In [ ]:
no_missing = df_missing_values[df_missing_values['Percentage_Missing'] == 0]
# skip index
for feat in no_missing[1:].index:
    info = feat_info[feat_info['attribute'] == feat]['information_level'].values
    print(f'{feat} is of information_type: {info}')

All features that have no missing values are of type person or other. ``Other`` is the information_level I added to those features that had no information available. It's possible that I have classified some features wrong which makes it fairly possible that all features that have no missing values are actually of type person.

In [ ]:
# drop features with 30 % or or missing values
azdias_feat_dropped = azdias_parsed.drop(features_top_missing, axis=1)
azdias_feat_dropped.shape

### 1.1.4 Removing Rows

In [ ]:
fig = plt.figure(figsize=(20,7))
azdias_feat_dropped.isnull().sum(axis=1).hist(bins=20)
plt.title('Missing Values per Row');
plt.xlabel('# Features')
plt.ylabel('# Rows');

I'll use a threshold of 30 % which means that rows that have more than 90 features missing will be treated as outliners and dropped. These rows will be placed in a separate dataset.

In [ ]:
azdias_low_missing, azdias_high_missing = split_df(azdias_feat_dropped, 210)

azdias_low_missing.shape, azdias_high_missing.shape

In [ ]:
print(f'Dataset with small amount of missing values per row has {azdias_low_missing.shape[0]} rows')
print(f'Dataset with large amount of missing values per row has {azdias_high_missing.shape[0]} rows')
print(f'New dataset contains {azdias_low_missing.shape[0] / azdias_feat_dropped.shape[0] * 100} % of the original amount of rows')

In [ ]:
columns = ['ZABEOTYP', 'SEMIO_TRADV', 'FINANZ_VORSORGER', 'FINANZTYP', 'ALTERSKATEGORIE_GROB', 'ONLINE_AFFINITAET']

for column in columns:
    compare_columns([azdias_low_missing, azdias_high_missing], column)

Even though the dataset with a high amount of missing values only makes up around 12 % of the total population we should pay close attention to this group. As shown above the distribution for some features are different between the two groups, which suggest that these groups are quite different from each other.

From here on I'll focus the analysis on the group with the smaller amount of missing values.

### 1.1.5 Feature Engineering

Handeling missing values is not the only preprocessing that needs to be done. Since the unsupervised learning algorithms that will be used in the customer segmentation report only work on data that is encoded numerically, all none numerical values must be encoded to numerical. Even though almost all of the values are numerical some of them don't represent numeric values (categories).

The type column in the feature file will be used as guidance in respect to each feature type:
- Interval and numeric features will be left as they are
- Most of the variables in the dataset are ordinal in nature. While ordinal values may technically be non-linear in spacing, make the simplifying assumption that the ordinal variables can be treated as being interval in nature (that is, kept without any changes).
- Special handling may be necessary for the remaining two variable types: categorical, and 'mixed'.

In [ ]:
feat_info_300 = feat_info[feat_info['attribute'].isin(azdias_low_missing.columns)].copy()
feat_info_300['type'].value_counts()

In [ ]:
obj_features = azdias_low_missing.select_dtypes(include=['object']).columns
for feat in obj_features:
    feat_type = feat_info_300[feat_info_300['attribute'] == feat]['type'].values
    if feat_type not in ['mixed', 'categorical']:
        print(feat)

In [ ]:
feat_info_300.loc[feat_info_300['attribute'] == 'D19_LETZTER_KAUF_BRANCHE', 'type'] = 'categorical'
feat_info_300.loc[feat_info_300['attribute'] == 'EINGEFUEGT_AM', 'type'] = 'mixed'

D19_LETZTER_KAUF_BRANCHE and EINGEFUEGT_AM are both features that are of type object. I've decided to classifiy D19_LETZTER_KAUF_BRANCHE as categorical and EINGEFUEGT_AM as mixed and they will be re-encoded with their corresponding type down below.

In [ ]:
types = get_feature_types(azdias_low_missing, feat_info_300)

In [ ]:
print(f"Remaining categorical features in the dataset:\n {types['categorical']}")

In [ ]:
binary, multiple = categorical_info(azdias_low_missing, types['categorical'])
print('Binary Features:')
print('\n'.join('Column: {} Values: {}'.format(k, v) for k, v in binary.items()))
print('-' * 35)
print('Multi-Level Features:')
print('\n'.join('Column: {} Values: {}'.format(k, v) for k, v in multiple.items()))

In [ ]:
for k,v in multiple.items():
    print(k)

In [ ]:
print(f"Remaining mixed features in the dataset:\n {types['mixed']}")

As stated above the ordinal and interval features will be treated as raw numeric values meaning they will be left alone. With regards to the categorical and mixed features I've decided to do the following:

- ``OST_WEST_KZ`` is the only binary categorical feature that needs to be re-encoded.
- ``WOHNLAGE`` will be treated as categorical and re-encoded with dummy.
- ``CAMEO_DEU_2015`` and ``CAMEO_DEUG_2015`` are both describing the wealth and life stage topology but at different scales. I've decided to keep ``CAMEO_DEUG_2015`` which describes the information at a rough scale and drop ``CAMEO_DEU_2015``. Another reason for dropping this feature is that it contains over 40 categories.
- ``CAMEO_INTL_2015`` combines information on two axes: wealth and life stage. Out of this feature two new ordinal features will be created. This will be achieved by splitting the original feature by it's tens- and ones-place.
- ``PRAEGENDE_JUGENDJAHRE`` combines information on three dimensions: generation by decade, movement (mainstream vs. avantgarde), and nation (east vs. west). While there aren't enough levels to disentangle east from west, two new variables will be created to capture the other two dimensions: an interval-type variable for decade, and a binary variable for movement.
- ``EINGEFUEGT_AM`` is in date format and will be re-encoded to capture the year.  
- ``PLZ8_BAUMAX`` will be treated as interval (no change)
- ``KBA05_HERSTTEMP`` and ``KBA05_MODTEMP`` will also be treated as interval (no change)
- ``LP_LEBENSPHASE_FEIN`` and ``LP_LEBENSPHASE_GROB`` will be dropped due to being high dimensional and to hard to separate in a good way.
- ``LNR`` is some kind of id/index and will be dropped.

I'll create a clean_data function that will be used to apply the re-enconding above and 

In [12]:
azdias_cleaned = clean_data(azdias, feat_info)

****** Step 1 - Load *****
Preparing to clean dataset with shape: (891221, 366)

Dropped these features from dataframe and feat_info: 
['LNR', 'LP_LEBENSPHASE_FEIN', 'LP_LEBENSPHASE_GROB', 'CAMEO_DEU_2015']
Shape after Step 1 - Load: (891221, 362)

****** Step 2 - Convert NaN codes *****


[------------------------------------------------------------------------] 100%


Shape after Step 2 - Convert NaN codes: (891221, 362)

****** Step 3 - Drop Columns with Missing values >= 30 % *****
Shape after Step 3 - Drop Columns with Missing values >= 30 %: (891221, 296)

****** Step 4 - Drop Rows with Missing values >= 30 % *****
Shape after Step 4 - Drop Rows with Missing values >= 30 %: (785215, 296)

****** Step 5 - Impute Missing Values *****


[------------------------------------------------------------------------] 100%


Shape after Step 5 - Impute Missing Values: (785215, 296)

AM NULL: 0
****** Step 6 - Re-encode Binary Categorical Features *****
Shape after Step 6 - Re-encode Binary Categorical Features: (785215, 296)

****** Step 7 - Re-encode Multi-Level Categorical Features *****
Shape after Step 7 - Re-encode Multi-Level Categorical Features: (785215, 422)

****** Step 8 - Re-encode Mixed Features *****
Shape after Step 8 - Re-encode Mixed Features: (785215, 424)



In [ ]:
#azdias_cleaned.to_pickle('data/azdias_cleaned.pkl')

In [ ]:
customers_cleaned = clean_data(customers, feat_info, azdias_cleaned.columns, customer_data=True)

In [ ]:
#customers_cleaned.to_pickle('data/customers_cleaned.pkl')

## Part 2: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

In [ ]:
# load pickle files
azdias_cleaned = pd.read_pickle('data/azdias_cleaned.pkl')
customers_cleaned = pd.read_pickle('data/customers_cleaned.pkl')

azdias_cleaned.shape, customers_cleaned.shape

### 2.1 Feature Scaling
Before applying dimensional reduction techniques to the data, feature scaling needs to be done so that the principal component vectors are not influenced by the natural differences in scale for features.

In [ ]:
scaler = StandardScaler()

# fit transform
scaled = scaler.fit_transform(azdias_cleaned.astype(float))

azdias_scaled = pd.DataFrame(data=scaled, index=azdias_cleaned.index, columns=azdias_cleaned.columns)
print("original shape:   ", azdias_cleaned.shape)
print("scaled shape:", azdias_scaled.shape)

In [ ]:
#scaler_file = 'models/scaler.save'
#joblib.dump(scaler, scaler_file)

### 2.2.1 Perform Dimensionality Reduction

With the data scaled, we are now ready to apply dimensionality reduction techniques.

- Sklearn's PCA class will be used to perform principal component analysis on the data, thus finding the vectors of maximal varaiance in the data. To start with all the features will be kept (so there's enough features to see the general trend in variability).
- Both the ratio of variance explained by each component as well as the cumulative variance explained will be plotted.
- Re-evaluate the number of components to keep and then re-fit the transform the data.

In [ ]:
# apply PCA
pca = PCA()

# fit transform
azdias_pca = pca.fit_transform(azdias_scaled)

In [ ]:
# Investigate the variance accounted for by each principal component.
variance = pca.explained_variance_ratio_

cumsum_variance=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=3)*100)

In [ ]:
n_components = len(pca.explained_variance_ratio_)
index = np.arange(n_components)
vals = pca.explained_variance_ratio_

plt.figure(figsize=(13,15))
plt.subplot(2, 1, 1)
plt.bar(index, cumsum_variance)
plt.ylabel('% Cumulative Explained Variance')
plt.xlabel('Number of Components')
plt.xticks(np.linspace(0,500, 10, endpoint=False))
plt.title('PCA Analysis')

# 15 components
plt.hlines(y=30, xmin=0, xmax=15, color='red', linestyles='-',zorder=1)
plt.vlines(x=15, ymin=0, ymax=30, color='red', linestyles='-',zorder=2)

# 100 components
plt.hlines(y=61, xmin=0, xmax=100, color='red', linestyles='-',zorder=3)
plt.vlines(x=100, ymin=0, ymax=61, color='red', linestyles='-',zorder=4)

# 200
plt.hlines(y=81, xmin=0, xmax=200, color='red', linestyles='-',zorder=5)
plt.vlines(x=200, ymin=0, ymax=81, color='red', linestyles='-',zorder=6)

plt.subplot(2, 1, 2)
plt.bar(index, vals)
plt.xticks(np.linspace(0,500, 10, endpoint=False))
plt.xlabel('Number of Components')
plt.ylabel('% Explained Cariance')
plt.title('PCA Analysis');

The plots above show that the explained variance plateaus a couple of times. The first and sharpest drop-off is around 15 components. At 15 componenets around 30 % of the total variance is explained. After this we can observe two other, but not as steep drop-offs around 100 and 200 components. At 100 components 61 % of the total variance is explained and at 200 82 %.

With each consecutive component less and less variance is explained which means that after a while the amount of additional explained variance probably is not worth the extra number of components.

With that said 100 components could be considered a good middle point.

In [ ]:
n_components = 100
print(f'{n_components} has a cumulative explained variane of: {sum(pca.explained_variance_ratio_[:n_components])}')

In [ ]:
# re-fit with n components
# apply PCA
pca = PCA(n_components=n_components)

# fit transform
azdias_pca = pca.fit_transform(azdias_scaled)

In [ ]:
#pca_file = 'models/pca.save'
#joblib.dump(pca, pca_file)

### 2.2.2 Interpret Principal Components
Now that we have our transformed principal components, it's a good idea to check out the weight of each variable on the first few components to see if they can be interpreted in some fashion.

Each principal component is a unit vector that points in the direction of highest variance (after accounting for the variance captured by earlier principal components). The further a weight is from zero, the more the principal component is in the direction of the corresponding feature. If two features have large weights of the same sign (both positive or both negative), then increases in one tend expect to be associated with increases in the other. To contrast, features with different signs can be expected to show a negative correlation: increases in one variable should result in a decrease in the other.

In [ ]:
dim1 = interpret_pca(azdias_scaled, pca, 0)
display_all(dim1)

In [ ]:
plot_pca(dim1, 1)

In [ ]:
dim2 = interpret_pca(azdias_scaled, pca, 1)
display_all(dim2)

In [ ]:
plot_pca(dim2, 2)

In [ ]:
dim3 = interpret_pca(azdias_scaled, pca, 2)
display_all(dim3)

In [ ]:
plot_pca(dim3, 3)

### 2.3 Clustering

### 2.3.1 Apply Clustering to General Population

In [ ]:
%%time
# Number of different cluster counts
n_clusters = np.arange(2,31)

# run k-means clustering on the data and compute the average within-cluster distances.
scores = [MiniBatchKMeans(i).fit(azdias_pca).score(azdias_pca) for i in n_clusters]     

In [ ]:
# elbow method
plt.ylabel('SSE')
plt.xlabel('# Clusters')
plt.title('SSE vs Clusters')
plt.plot(n_clusters, np.abs(scores), linestyle='-', marker='o', color='red');

In [ ]:
%%time
# re-fith with 13 clusters
kmeans = KMeans(n_clusters=10, random_state=42)
#kmeans = KMeans(n_clusters=11, random_state=42, n_jobs=-1)

# general population predictions
azdias_predictions = kmeans.fit_predict(azdias_pca)

In [ ]:
#kmeans_file = 'models/kmeansstrange.save'
#joblib.dump(kmeans, kmeans_file)

In [ ]:
# apply scaler, pca and kmeans
customers_scaled = scaler.transform(customers_cleaned.astype('float'))
customers_pca = pca.transform(customers_scaled)
customers_preds = kmeans.predict(customers_pca)

In [ ]:
azdias_predictions = clusters_predict(azdias_cleaned)
customers_preds = clusters_predict(customers_cleaned)

In [ ]:
scaler = joblib.load('models/scaler.save')
pca = joblib.load('models/pca.save')
kmeans = joblib.load('models/kmeans.save')

### 2.3.2 Compare Customer Data to Demographics Data 

At this point, we have clustered data based on demographics of the general population of Germany, and seen how the customer data for a mail-order sales company maps onto those demographic clusters. Let's end this section with a comparison of the two cluster distributions to see where the strongest customer base for the company is.

In [ ]:
#general_proportions = Counter(azdias_predictions['Clusters'])
#customers_proportions = Counter(customers_preds['Clusters'])

general_proportions = Counter(azdias_predictions)
customers_proportions = Counter(customers_preds)

In [ ]:
for k, v in general_proportions.items():
    general_proportions[k] = v / len(azdias)
    
for k, v in customers_proportions.items():
    customers_proportions[k] = v / len(customers)

In [ ]:
df_props = pd.DataFrame(columns=["Dataset", "Cluster", "Proportions"])
for k, v in customers_proportions.items():
    df_props = df_props.append({"Dataset" : "customer", "Cluster": k, "Proportions": v}, ignore_index=True)

for k, v in general_proportions.items():
    df_props = df_props.append({"Dataset" : "general", "Cluster": k, "Proportions": v}, ignore_index=True)

In [ ]:
fig = plt.figure(figsize=(15,7))
sns.barplot(x="Cluster", y="Proportions", hue="Dataset", data=df_props);

In [ ]:
center_target = kmeans.cluster_centers_[5]
center_general = kmeans.cluster_centers_[4]

center_target.shape, center_general.shape

In [ ]:
# Due to large scale in some of the features no reversing of the scaler is applied.
# Would skew the plot
center_target_inv = pca.inverse_transform(center_target)
center_general_inv = pca.inverse_transform(center_general)

In [ ]:
features = azdias_cleaned.columns

plt.figure(figsize=(10,100))

sns.barplot(x=center_target_inv, y=features, label="C5 Target", color="#5499C7")
sns.barplot(x=center_general_inv, y=features, label="C4 General", color="#E59866")
          
plt.legend(loc="upper left");

In [ ]:
def plot_comparison(column, customer, general):
    sns.set(style="darkgrid")
    fig, (ax1, ax2) = plt.subplots(figsize=(12,4), ncols=2)
    sns.countplot(x = column, data=customer, ax=ax1, palette="Set3")
    ax1.set_xlabel('Value')
    ax1.set_title('Distribution of target population')
    sns.countplot(x = column, data=general, ax=ax2, palette="Set3")
    ax2.set_xlabel('Value')
    ax2.set_title('Distribution of non-target population')
    fig.tight_layout()
    plt.show()

In [ ]:
df_preds_azdias = pd.DataFrame(azdias_predictions['Clusters'].values, columns = ['Cluster'])
df_preds_customers = pd.DataFrame(customers_preds['Clusters'].values, columns = ['Cluster'])

In [ ]:
target = df_preds_customers[df_preds_customers['Cluster'] == 5].index
general = df_preds_azdias[df_preds_azdias['Cluster'] == 4].index

df_target = customers_cleaned.iloc[target]
df_general = azdias_cleaned.iloc[general]

In [ ]:
plot_comparison('HH_EINKOMMEN_SCORE', df_target, df_general)

In [ ]:
plot_comparison('KKK', df_target, df_general)

In [ ]:
plot_comparison('MOBI_REGIO', df_target, df_general)

In [ ]:
plot_comparison('ANREDE_KZ', df_target, df_general)

In [ ]:
plot_comparison('EWDICHTE', df_target, df_general)

### 2.3.3 Discussing Compare Customer and Demographics Data

Since there are over 400 features in the dataset it's impracticable to compare the datasets exhaustively. Therefore, I've decided to compare five features with the purpose to illustrate a point.

- ``HH_EINKOMMEN_SCORE``: estimated household net income (lower = higher income)
- ``KKK``: purchasing power (lower = higher purchasing power)
- ``MOBI_REGIO``: moving patterns (lower = higher mobility)
- ``ANREDE_KZ``: gender (1 = male & 2 = female)
- ``EWDICHTE``: density of inhabitants per square kilometer (lower = less dense)

From the tables above we can observe that the customer population have a higher estimated household net income and higher purchasing power. Customers are less mobile, a larger proportion are male and are living in less dense areas.

## Part 3: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [13]:
mailout_train = pd.read_csv('data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';', dtype={18:'str',19:'str'})
mailout_test = pd.read_csv('data/Udacity_MAILOUT_052018_TEST.csv', sep=';', dtype={18:'str',19:'str'})

mailout_train.shape, mailout_test.shape

((42962, 367), (42833, 366))

In [136]:
#mailout_train.to_pickle('data/mailout_train.pkl')
#mailout_test.to_pickle('data/mailout_test.pkl')

mailout_train = pd.read_pickle('data/mailout_train.pkl')
mailout_test = pd.read_pickle('data/mailout_test.pkl')
mailout_train.shape, mailout_test.shape

((42962, 367), (42833, 366))

In [15]:
target = mailout_train['RESPONSE']
mailout_train.drop('RESPONSE', axis=1, inplace=True)

In [16]:
mailout_train_cleaned = clean_data(mailout_train, feat_info, columns=azdias_cleaned.columns, drop_rows=False)
mailout_test_cleaned = clean_data(mailout_test, feat_info, columns=azdias_cleaned.columns, drop_rows=False)

****** Step 1 - Load *****
Preparing to clean dataset with shape: (42962, 366)

Dropped these features from dataframe and feat_info: 
['LNR', 'LP_LEBENSPHASE_FEIN', 'LP_LEBENSPHASE_GROB', 'CAMEO_DEU_2015']
Shape after Step 1 - Load: (42962, 362)

****** Step 2 - Convert NaN codes *****


[------------------------------------------------------------------------] 100%


Shape after Step 2 - Convert NaN codes: (42962, 362)

****** Step 3 - Drop Columns with Missing values >= 30 % *****


Shape after Step 3 - Drop Columns with Missing values >= 30 %: (42962, 296)

****** Step 4 - Drop Rows with Missing values >= 30 % *****
Shape after Step 4 - Drop Rows with Missing values >= 30 %: (42962, 296)

****** Step 5 - Impute Missing Values *****


[------------------------------------------------------------------------] 100%


...

[------------------------------------------------------------------------] 100%


Shape after Step 2 - Convert NaN codes: (42833, 362)

****** Step 3 - Drop Columns with Missing values >= 30 % *****


Shape after Step 3 - Drop Columns with Missing values >= 30 %: (42833, 296)

****** Step 4 - Drop Rows with Missing values >= 30 % *****
Shape after Step 4 - Drop Rows with Missing values >= 30 %: (42833, 296)

****** Step 5 - Impute Missing Values *****


[------------------------------------------------------------------------] 100%


Shape after Step 5 - Impute Missing Values: (42833, 296)

AM NULL: 0
****** Step 6 - Re-encode Binary Categorical Features *****
Shape after Step 6 - Re-encode Binary Categorical Features: (42833, 296)

****** Step 7 - Re-encode Multi-Level Categorical Features *****
Shape after Step 7 - Re-encode Multi-Level Categorical Features: (42833, 421)

****** Step 8 - Re-encode Mixed Features *****
Shape after Step 8 - Re-encode Mixed Features: (42833, 423)

****** Step 8 - Add Missing Columns *****
Shape after Step 8 - Add Missing Columns: (42833, 424)



In [ ]:
sns.countplot(target).set_title("Distribution of Label");

As shown by the table above the dataset is imbalanced: there are far more observations where the person has not responded (0) than responsed (1). Therefore, accuracy is a poor mertric to use. The Receiver Operating Characteristic Area Under the Curve (ROC AUC) will be used instead. Randomly guessing will lead to a ROC AUC of 0.5 and a perfect score would be 1.0.

In [141]:
# create train and validation dataset.
# use stratify to make sure same proportions of class labels as the input dataset

SEED = 28

X_train, X_test, y_train, y_test = train_test_split(mailout_train_cleaned.astype('float'),
                                                    target,
                                                    test_size=0.2,
                                                    random_state=SEED,
                                                    stratify = target.values)

The validation set will be used to validate the model. This part of the data will not be used to train on when tuning hyperparameters.

### 3.1 Approach and Default Models

Depending on what the initial analysis show I may try to get a better score by creating a stacked model (ml-ens). This approach will hopefully lead to a better prediction. In addition to this I'll use the skopt library to tune the hyperparameters (Bayesian optimization).

In [126]:
# 5 stratified folds
skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)
skf.get_n_splits(mailout_train_cleaned, target)

5

In [45]:
def create_base_models():
    basedModels = []
    basedModels.append(('LR', LogisticRegression(solver='liblinear', random_state=SEED)))
    basedModels.append(('RF', RandomForestClassifier(n_estimators=100, random_state=SEED)))
    basedModels.append(('XGB', xgb.XGBClassifier(random_state=SEED)))
    basedModels.append(('LGBM', lgb.LGBMClassifier(random_state=SEED)))
    basedModels.append(('GB', GradientBoostingClassifier(random_state=SEED)))
    basedModels.append(('MLP', MLPClassifier(random_state=SEED)))
    
    return basedModels

In [122]:
def evaluate(features, target, models):
    results = []
    names = []
    for name, model in models:
        cv_results = cross_val_score(model, X_train, y_train, cv=skf, scoring='roc_auc', n_jobs=-1)
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
        
    return names, results

In [123]:
def get_scaled_preprocess(type_of_scaler):
    
    if type_of_scaler == 'standard':
        scaler = StandardScaler()
    elif type_of_scaler == 'minmax':
        scaler = MinMaxScaler()
        
    pipelines = []
    pipelines.append((type_of_scaler+'LR', Pipeline([('Scaler', scaler), ('LR', LogisticRegression(solver='liblinear', random_state=SEED))])))
    pipelines.append((type_of_scaler+'RF', Pipeline([('Scaler', scaler), ('RF', RandomForestClassifier(n_estimators=100, random_state=SEED))])))   
    pipelines.append((type_of_scaler+'XGB', Pipeline([('Scaler', scaler), ('XGB', xgb.XGBClassifier(random_state=SEED))])))
    pipelines.append((type_of_scaler+'LGBM', Pipeline([('Scaler', scaler), ('LGBM', lgb.LGBMClassifier(random_state=SEED))])))
    pipelines.append((type_of_scaler+'GB', Pipeline([('Scaler', scaler), ('GB', GradientBoostingClassifier(random_state=SEED))])))   
    pipelines.append((type_of_scaler+'MLP', Pipeline([('Scaler', scaler), ('MLP', MLPClassifier(random_state=SEED))])))   

    
    return pipelines

In [124]:
def create_score_df(names, results):
    def floatingDecimals(f_val, dec=3):
        prc = "{:."+str(dec)+"f}" 
    
        return float(prc.format(f_val))

    scores = []
    for r in results:
        scores.append(floatingDecimals(r.mean(),4))

    scoreDataFrame = pd.DataFrame({'Model':names, 'Score': scores})
    return scoreDataFrame

Let's start by checking how the models perform on unscaled, standard scaled and minmax scaled data.

In [125]:
#unscaled 
models = create_base_models()
names, results = evaluate(mailout_train_cleaned, target, models)
baseline_score = create_score_df(names, results)
baseline_score

LR: 0.665530 (0.036172)
RF: 0.597039 (0.022422)
XGB: 0.755744 (0.029427)
LGBM: 0.707587 (0.013561)
GB: 0.759239 (0.022813)
MLP: 0.607565 (0.056376)


,Model,Score
0,LR,0.6655
1,RF,0.5970
2,XGB,0.7557
3,LGBM,0.7076
4,GB,0.7592
5,MLP,0.6076


In [50]:
# standard scaler
models = get_scaled_preprocess('standard')
names, results = evaluate(X_train, y_train, models)
standard_scaler_score = create_score_df(names, results)

df_score = pd.concat([baseline_score, standard_scaler_score], axis=1)
df_score

standardLR: 0.624120 (0.026171)
standardRF: 0.562346 (0.021686)
standardXGB: 0.760572 (0.027826)
standardLGBM: 0.689190 (0.018108)
standardGB: 0.752448 (0.015501)
standardMLP: 0.574412 (0.036794)


,Model,Score,Model,Score
0,LR,0.6385,standardLR,0.6241
1,RF,0.5743,standardRF,0.5623
2,XGB,0.7589,standardXGB,0.7606
3,LGBM,0.7005,standardLGBM,0.6892
4,GB,0.7481,standardGB,0.7524
5,MLP,0.5790,standardMLP,0.5744


In [51]:
# minmax scaler
models = get_scaled_preprocess('minmax')
names, results = evaluate(X_train, y_train, models)
minmax_scaler_score = create_score_df(names, results)

df_score = pd.concat([df_score, minmax_scaler_score], axis=1)
df_score

minmaxLR: 0.652879 (0.016759)
minmaxRF: 0.576846 (0.036397)
minmaxXGB: 0.760824 (0.024929)
minmaxLGBM: 0.694179 (0.020771)
minmaxGB: 0.753639 (0.018721)
minmaxMLP: 0.576122 (0.024516)


,Model,Score,Model,Score,Model,Score
0,LR,0.6385,standardLR,0.6241,minmaxLR,0.6529
1,RF,0.5743,standardRF,0.5623,minmaxRF,0.5768
2,XGB,0.7589,standardXGB,0.7606,minmaxXGB,0.7608
3,LGBM,0.7005,standardLGBM,0.6892,minmaxLGBM,0.6942
4,GB,0.7481,standardGB,0.7524,minmaxGB,0.7536
5,MLP,0.5790,standardMLP,0.5744,minmaxMLP,0.5761


When comparing the result between no scaling, standardscaler and minmaxscaler we can see that minmaxscaler provides the best result. From here on now I'll use minmaxscaler to scale the data.

The best performing model by a small margin is the XGBClassifier. This will be our base model and the next step is to see if we can improve the score by tuning the hyperparameters.

In [53]:
def create_pipeline(clf):
    pipeline = Pipeline([
        ('scaler', MinMaxScaler()),
        ('clf', clf)
    ])
    
    return pipeline

In [69]:
scaler = MinMaxScaler()
scaler.fit(mailout_train_cleaned.astype('float'))

mailout_train_cleaned_scaled = scaler.transform(mailout_train_cleaned)

In [154]:
# create minmax scaled train and validation set
X_train2, X_test2, y_train2, y_test2 = train_test_split(mailout_train_cleaned_scaled,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=SEED,
                                                    stratify = target.values)

In [164]:
xgb_0 = xgb.XGBClassifier(random_state=SEED)
xgb_0.fit(X_train2, y_train2)
preds = xgb_0.predict_proba(X_test2)[:,1]

Base XGBClassifier score on validation set: {roc_auc_score(y_test2, preds)}


In [165]:
print(f'Base XGBClassifier score on validation set: {roc_auc_score(y_test2, preds)}')

Base XGBClassifier score on validation set: 0.7717908188105672


## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')